# Введение

На практике встречается задача Sequential Recommendation (SR): нужно предсказать последовательность наиболее вероятных в покупке категорий товаров (токенов), на основе предыдущей последовательности категорий товаров. Чаще всего её можно увидеть в сфере маркетинга, банкинга, когда мы хотим узнать последовательность действий клиента, на основе его предыдущей активности. В этой работе изучаются современные методы решения задачи SR, изучена их эффективность на [задаче  Альфа-Банка](https://www.kaggle.com/competitions/alfabank-campus/). В ней на основе категорий покупок пользователя в течение текущего месяца требовалось предсказать последовательность из десяти категорий покупок в следующем месяце.


Задача — предсказать следующие 10 [MCC-кодов](https://www.banki.ru/wikibank/mcc-kod/) 7033 клиентов, основываясь на их предыдущих тратах.

В качестве тренировочной выборки предоставлены последовательности MCC-кодов других 7033 клиентов с таргетом в виде 10 последующих MCC-кодов.

Целевая метрика — [MAP@10](https://habr.com/ru/company/econtenta/blog/303458/
)
В итоге для обучения модели я подменил задачу: предсказывал не 10 последующих MCC-кодов авторегрессионно, а предсказывал 10 наиболее вероятных кандидатов на позицию следующего MCC-кода. Авторегрессионный подход показывал результат значительно хуже.

Сперва было опробовано свести оптимизацию *MAP@10* к *NLP*-задаче Language Modelling, рассматривая предыдущие коды как контекст, а следующие 10 кодов для предсказания - маски.

На самом деле эта идея уже была опробована другими и имеет отдельную [научную статью](https://arxiv.org/abs/1511.06939) 2015-го года о *session-based recommendations*. В ней авторы предлагают использовать рекуррентные нейронные сети (*RNN*) с механизмом *GRU* (*gated recurrent unit*) для задачи рекомендаций.

Именно они изобрели модель *GRU4Rec*, ставшей повсеместным бейзлайном для последовательных рекомендаций (*sequential recommendations*). Эта модель и была опробована в текущей работе.

На [*paperswithcode*](https://paperswithcode.com/task/sequential-recommendation) есть готовая реализацию *GRU4Rec* в библиотеке [*RecBole*](https://recbole.io/docs/index.html).

*RecBole* оказался полезным инструментов, позволивший попробовать и другие *state of the art* модели. Среди них были:

- [*BERT4Rec*](https://recbole.io/docs/user_guide/model/sequential/bert4rec.html) — трансформер, адаптированный под sequential recommendation.
- [*RepeatNet*](https://recbole.io/docs/user_guide/model/sequential/repeatnet.html) — RNN, эксплицитно учитывающая феномен *repeat consumption* в составлении рекомендации.
- [*Caser*](https://recbole.io/docs/user_guide/model/sequential/caser.html) и [*RepeatItNet*](https://recbole.io/docs/user_guide/model/sequential/nextitnet.html), использующие свёрточные слои (как в задачах CV).

Однако лучшие результаты в экспериментах среди них показал всё тот же *GRU4Rec*.

Допускаю, что это связано не с применимостью моделей на этом датасете, а скорее временными лимитами на использование *GPU* в *Colab Pro* — перечисленные выше модели тяжелее и требуют бОльшего времени и ресурсов на обучение и подбор оптимальных гиперпараметров.

Ознакомиться с примерным временем обучения моделей библиотеки можно на [этой странице](https://github.com/RUCAIBox/RecBole/blob/master/asset/time_test_result/Sequential_recommendation.md).

# Импортирование библиотек

In [ ]:
!pip install recbole
!pip install torch
!pip install ray

In [ ]:
import pandas as pd
import numpy as np
import os
# Для вывода работы библиотеки RecBole
import logging
from logging import getLogger

# Для типизации функций
from typing import List, Tuple

from collections import defaultdict
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_logger
from recbole.utils.case_study import full_sort_topk
import torch

# Для того чтобы тетрадка запускалась и на CPU, и на GPU
DEVICE = torch.device("cuda") if torch.cuda.is_available()\
else torch.device("cpu")

#from google.colab import drive
#drive.mount('/content/drive')

# Обработка данных

Для обучения модели я решил объединить тренировочную и тестовую выборки. Хоть в тренировочной и содержатся последовательности для других клиентов, в них всё-таки может содержаться полезная информация об айтемах.

In [ ]:
FILE_PATH_TRAIN = \
'/content/drive/MyDrive/colab_notebooks/alfa_data/df_train.csv'
FILE_PATH_TEST = \
'/content/drive/MyDrive/colab_notebooks/alfa_data/df_test.csv'

df_train = pd.read_csv(FILE_PATH_TRAIN, sep=';')
df_test = pd.read_csv(FILE_PATH_TEST, sep=';')

In [ ]:
display(df_train.head())
display(df_test.head())

,Id,Data,Target
0,0,"4814,4814,6010,6011,4814,6011,6011,4814,6011,6...","4814,4814,4814,4814,5411,4814,4814,4814,4814,4814"
1,1,"6011,6011,6011,6011,6011,6011,6011,4814,4814,4...","4814,6011,4814,6011,4814,4814,6011,4814,6011,4814"
2,2,"8021,6011,6011,6010,4829,4814,6011,6011,6011,6...","6011,6011,6010,4829,4829,6010,6011,6011,4814,6011"
3,3,"4814,6011,4814,4814,4814,6011,6011,5691,5691,5...","6011,6011,6010,6011,6011,4814,4814,6011,4814,4814"
4,4,"4814,4814,4814,4814,4814,4814,5946,4814,4814,6...","5499,6011,4814,4829,5200,5411,5499,5912,5411,5912"


,Id,Data
0,0,"4814,4814,6011,6011,6010,6011,6011,4814,6011,4..."
1,1,"6010,6011,6010,5411,5411,5977,6011,6010,5411,6..."
2,2,"4814,6011,5251,6011,7832,5641,5814,4829,5311,6..."
3,3,"6011,4722,4722,4722,4814,6011,6011,4829,6011,6..."
4,4,"4814,4814,4814,6011,4814,4814,4814,4814,4814,4..."


Переводим строки в списки строк:

In [ ]:
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()

convert_to_arr = lambda s: s.split(',')
df_train['data'] = df_train['data'].apply(convert_to_arr)
df_train['target'] = df_train['target'].apply(convert_to_arr)
df_test['data'] = df_test['data'].apply(convert_to_arr)

In [ ]:
display(df_train.head())
display(df_test.head())

,id,data,target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."


,id,data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


Объединяем столбцы `data` и `target` в тренировочном датасете:

In [ ]:
train_data = df_train.apply(lambda row: row[1] + row[2], axis=1).reset_index()
test_data = df_test.copy()

In [ ]:
train_data.columns = ['user_id', 'item_ids']
test_data = test_data.rename({'id': 'user_id', 'data': 'item_ids'}, axis=1)
display(train_data.head())
display(test_data.head())

,user_id,item_ids
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481..."


,user_id,item_ids
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


Перед объединением присваиваем новые `user_id` клиентам из тренировочного датасета:

In [ ]:
train_data['user_id'] = range(7033, 7033 * 2)
train_data.head()

,user_id,item_ids
0,7033,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481..."
1,7034,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481..."
2,7035,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601..."
3,7036,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569..."
4,7037,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481..."


Конкатенируем датафреймы в один для обучения модели и предсказаний:

In [ ]:
full_train = pd.concat([test_data, train_data])

# Проверяем, что строк в новом датафрейме вдвое больше, чем в train
assert full_train.shape[0] == train_data.shape[0] * 2

display(full_train.head())
full_train.tail()

,user_id,item_ids
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


,user_id,item_ids
7028,14061,"[6010, 4829, 6011, 6011, 6011, 6010, 6011, 601..."
7029,14062,"[4814, 5699, 5641, 5411, 6010, 6011, 4814, 601..."
7030,14063,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 601..."
7031,14064,"[4814, 4814, 5411, 6011, 6011, 4814, 4814, 481..."
7032,14065,"[6011, 6011, 6011, 6011, 5541, 4814, 6011, 601..."


# Форматирование данных для *RecBole*

Используемая библиотека *RecBole* принимает для обучения файлы определённого формата — [*Atomic*](https://recbole.io/docs/user_guide/data/atomic_files.html).  

Для примера в документации *RecBole* представлен сэмпл корректного форматирования популярного *RecSys* датасета `ml-100k` в формате *Atomic*.

In [ ]:
EXAMPLE_FILE_PATH = \
'/content/drive/MyDrive/colab_notebooks/pytorch/ml-100k.inter'
ml_100k = pd.read_csv(EXAMPLE_FILE_PATH, sep='\t')

In [ ]:
display(ml_100k)

,user_id:token,item_id:token,rating:float,timestamp:float
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


То есть нужно будет развернуть данные по типу один *user_id* - одно наблюдение, а также добавить *timestamp* — последовательные рекомендательные модели типа *GRU4Rec* в библиотеке не работают без столбца со временем.

Вижу, что время в колонке `timestamp:float` представлено в секундах:

In [ ]:
raw_timestamp = ml_100k.loc[0, 'timestamp:float']
print(pd.to_datetime(raw_timestamp, unit='s'))

1997-12-04 15:55:49


Добавляю фиктивные `timestamp`-ы чтобы явно задать последовательность клиентских транзакций. У каждого клиента самая недавняя транзакция пусть будет в `timestamp` 150 000 000, а каждая предшествующая — по 6 часов (21 600 сек.) раньше:

In [ ]:
print(pd.to_datetime(150_000_0000, unit='s'))
print(f'21 600 секунд = {int(21_600 / 60 / 60)} часов.')

2017-07-14 02:40:00
21 600 секунд = 6 часов.


Оформляю для такого преобразования функцию:

In [ ]:
origin = 150_000_0000
period = 21_600

def add_timestamp(arr: List[str]) -> List[Tuple[str, int]]:
    '''
    Функция принимает на вход последовательный список с MCC-кодами,
    возвращает список с кортежами (MCC-код, timestamp)
    '''
    new_arr_rev = []
    for i, elem in enumerate(reversed(arr)):
        new_arr_rev.append((elem, origin - (i * period)))
    return new_arr_rev[::-1]

In [ ]:
full_train['item_timestamp'] = full_train['item_ids'].apply(add_timestamp)

In [ ]:
full_train = full_train[['user_id', 'item_timestamp']]

Преобразуем датафрейм по типу одна транзакция - одна строка:

In [ ]:
full_exploded = full_train.explode('item_timestamp').reset_index(drop=True)
full_exploded.head()

,user_id,item_timestamp
0,0,"(4814, 1495183200)"
1,0,"(4814, 1495204800)"
2,0,"(6011, 1495226400)"
3,0,"(6011, 1495248000)"
4,0,"(6010, 1495269600)"


Выделяем MCC-код и `timestamp` в отдельные столбцы:

In [ ]:
full_exploded['item_id'] = \
full_exploded['item_timestamp'].apply(lambda arr: arr[0])

full_exploded['timestamp'] = \
full_exploded['item_timestamp'].apply(lambda arr: arr[1])

full_exploded = full_exploded.drop('item_timestamp', axis=1)

In [ ]:
display(full_exploded.head())
full_exploded.shape

,user_id,item_id,timestamp
0,0,4814,1495183200
1,0,4814,1495204800
2,0,6011,1495226400
3,0,6011,1495248000
4,0,6010,1495269600


(6752236, 3)

Наконец, добавляем необходимую явную типизацию в названия колонок:

In [ ]:
full_exploded.columns = ['user_id:token', 'item_id:token', 'timestamp:float']
display(full_exploded.head())
display(full_exploded.tail())

,user_id:token,item_id:token,timestamp:float
0,0,4814,1495183200
1,0,4814,1495204800
2,0,6011,1495226400
3,0,6011,1495248000
4,0,6010,1495269600


,user_id:token,item_id:token,timestamp:float
6752231,14065,5411,1499913600
6752232,14065,6011,1499935200
6752233,14065,5541,1499956800
6752234,14065,6010,1499978400
6752235,14065,4814,1500000000


Сохраняем получившийся файл для дальнейшего обучения.  
Важный момент:  
*RecBole*-у важно, чтобы данные для обучения лежали в одной папке, а сами файлы (у нас он один) носили то же имя, что и папка, а также имели соответствующее типу данных расширение. У нас это `.inter` — файл с *interactions* `user_id` и `item_id`.

In [ ]:
new_folder = '/content/drive/MyDrive/colab_notebooks/alfa_data/full_train'

# Создаём новую папку 'full_train' если она не существует
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

new_csv = ('/content/drive/MyDrive/colab_notebooks/alfa_data/full_train/'
           + 'full_train.inter')

# Сохраняем csv с данными для обучения если ещё этого не делали
if not os.path.exists(new_csv):
    full_exploded[['user_id:token', 'item_id:token', 'timestamp:float']]\
    .to_csv(new_csv, sep='\t', index=False)


# Обучение модели

Диапазон подобранных гиперпараметров:

- `MAX_ITEM_LIST_LENGTH`: [20 - 150]
- `embedding_size`: [64 - 256]
- `hidden_size`: [64 - 256]
- `num_layers`: [1 - 3]
- `dropout_prob`: [0 - 0.5]
- `loss_type`: ['CE', 'BPR']
- `train_batch_size`: [256 - 7033]

В [оригинальной статье](https://arxiv.org/abs/1511.06939), представившей модель *GRU4Rec* авторы не используют эмбеддинги для айтемов (в их экспериментах *one-hot encoding* работал лучше), а также использовали функцию активации `tanh` для выходного слоя.

Пожалуй, наиболее важное различие — вместо *pointwise* кросс-энтропии авторы использовали *pairwis*e функции потерь *BPR* (*Bayesian Personalized Ranking*) и *TOP1*, опирающиеся на *negative sampling*.

Обучение модели с данными гиперпараметрами требует примерно 26 гигабайт System RAM, но потребление памяти можно сократить в разы и запустить код в обычном бесплатном рантайме *Google Colab* на CPU, уменьшив значения таких гиперпараметров как `MAX_ITEM_LIST_LENGTH`, `train_batch_size`, `hidden_size` и `num_layers`.

In [ ]:

config_dict = {
    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "load_col": {"inter": ["user_id", "item_id", "timestamp"]},
    "ITEM_LIST_LENGTH_FIELD": "item_length",
    "LIST_SUFFIX": "_list",
    "MAX_ITEM_LIST_LENGTH": 130,  # максимальная длина последовательности
    "embedding_size": 256, # размерность эмбеддингов
    "hidden_size": 256, # размерность скрытого состояния GRU
    "num_layers": 2,  # кол-во скрытых слоёв
    "dropout_prob": 0.3,  # droupout-вероятность в обучении
    "loss_type": "CE",  # функция потерь
    "epochs": 50,
    "train_batch_size": 4096,
    "eval_batch_size": 4096,
    "train_neg_sample_args": None, # не используем negative sampling
    # словарь с параметрами валидации
    "eval_args": {
        "group_by": "user", # группируем по пользователям
        "order": "TO", # сортируем по timestamp-у
        "split": {"LS": "valid_only"}, # leave-one-out validation
        "mode": "full", # используем все данные для inference
    },
    "metrics": ["Recall", "MRR", "NDCG", "Hit", "Precision", "MAP"],
    "topk": 10,
    "valid_metric": "MAP@10", # валидационная метрика, также целевая метрика
    # соревнования
    "data_path": "/content/drive/MyDrive/colab_notebooks/alfa_data/",
    "stopping_step": 2, # останавливаем обучение если в течение
    # этого кол-ва эпох не было улучшения валидационной метрики
    "device": DEVICE,
}


In [ ]:
# Код отсюда: https://www.kaggle.com/code/muddywaters23/recbole-gru4rec

config = Config(model='GRU4Rec', dataset='full_train', config_dict=config_dict)

# инициализируем логгеры для вывода информации
logger = getLogger()
logger.setLevel(logging.INFO)


c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)


logger.info(config)

INFO:root:
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /content/drive/MyDrive/colab_notebooks/alfa_data/full_train
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'group_by': 'user', 'order': 'TO', 'split': {'LS': 'valid_only'}, 'mode': 'full'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = MAP@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Pa

Создаём объекты тренировочной выборки и валдиацонной:

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, _ = data_preparation(config, dataset)

INFO:root:full_train
The number of users: 14067
Average actions of users: 480.0395279397128
The number of items: 185
Average actions of items: 36696.934782608696
The number of inters: 6752236
The sparsity of the dataset: -159.46237984625702%
Remain Fields: ['user_id', 'item_id', 'timestamp']
full_train
The number of users: 14067
Average actions of users: 480.0395279397128
The number of items: 185
Average actions of items: 36696.934782608696
The number of inters: 6752236
The sparsity of the dataset: -159.46237984625702%
Remain Fields: ['user_id', 'item_id', 'timestamp']
INFO:root:[Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
INFO:root:[Evaluation]: eval_batch_size = [4096] eval_args: [{'group_by': 'user', 'o

In [ ]:
# Если менять модель и перезапускать данную ячейку, рекомендую очищать кэш GPU
# при помощи этой строки:

# torch.cuda.empty_cache()

Инициализируем модель и обучаем.

Для полной репродукции эксперимента можно нервно смотреть на график использования RAM и надеяться, что рантайм не упадёт :)

In [ ]:
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# инициализируем "тренера" модели
trainer = Trainer(config, model)

# сохраняем лучшие результаты
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


INFO:root:GRU4Rec(
  (item_embedding): Embedding(185, 256, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(256, 256, num_layers=2, bias=False, batch_first=True)
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 899584
GRU4Rec(
  (item_embedding): Embedding(185, 256, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(256, 256, num_layers=2, bias=False, batch_first=True)
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 899584
INFO:root:epoch 0 training [time: 462.81s, train loss: 3301.9386]
epoch 0 training [time: 462.81s, train loss: 3301.9386]
INFO:root:epoch 0 evaluating [time: 0.45s, valid_score: 0.590900]
epoch 0 evaluating [time: 0.45s, valid_score: 0.590900]
INFO:root:valid result: 
recall@10 : 0.934    mrr@10 : 0.5909    ndcg@10 : 0.6748    hit@10 : 0.934    precision@10 : 0.

KeyboardInterrupt: ignored

# Предсказание для клиентов тестовой выборки

Складываем предсказания для клиентов тестовой выборки в массив `topk_items`:

Чтобы использовать больше информации для дальнейшего блендинга предсказаний разных моделей, приравниваем параметр `k` к 15 — так в список сложим 15 наиболее вероятных следующих MCC-кодов, а не 10.

In [ ]:
# Код отсюда:
# https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

topk_items = []
for internal_user_id in list(range(dataset.user_num))[1:7034]:
    _, topk_iid_list = full_sort_topk([internal_user_id],
                                      model,
                                      valid_data,
                                      k=15,
                                      device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(
        dataset.iid_field, last_topk_iid_list.cpu()
        ).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

7033


Пример предсказания для клиента `user_id` 0:

In [ ]:
topk_items[0]

['6011',
 '4829',
 '4814',
 '6010',
 '5411',
 '5499',
 '5541',
 '5912',
 '5331',
 '5921',
 '5812',
 '5533',
 '5999',
 '5983',
 '5814']

Форматируем предсказания в формат, требуемый соревнованием:

In [ ]:
test_preds = []
for arr in topk_items:
    test_preds.append(' '.join(arr))

In [ ]:
test_preds[:10]

['6011 4829 4814 6010 5411 5499 5541 5912 5331 5921 5812 5533 5999 5983 5814',
 '6011 5411 6010 4814 5912 5451 5541 5983 5331 5533 5977 4829 5499 5691 4812',
 '6010 6011 4829 5411 4814 5499 5814 5812 5200 5912 5331 5999 6012 5541 7832',
 '6011 4814 5411 4829 5912 5499 5814 5200 6010 6012 5812 5722 4812 5964 5977',
 '4814 6011 6010 4829 5411 5912 5541 5499 5331 6012 5983 6536 5661 4812 5977',
 '6011 5661 5411 5691 4814 5541 5999 5921 5977 5912 6010 5651 5812 5331 5261',
 '5499 6011 5411 4814 5999 5812 5921 5912 5945 4829 5331 5311 5691 6010 5814',
 '5411 4829 6011 5499 4814 5541 5977 5699 6010 5651 5912 5641 5814 5661 5691',
 '5411 4814 6011 5541 5499 5311 5211 6010 4829 5912 5261 5983 5814 5533 5331',
 '6010 4829 6011 4814 5411 5541 4900 5912 5945 5814 5261 4812 5691 5499 5812']

Выполняем код из ноутбука с бейзлайном и сохраняем csv-файл с предсказаниями:

In [ ]:
# Создайте DataFrame с предсказаниями
predictions_df = pd.DataFrame({'Id': range(len(test_preds)),
                               'Predicted': test_preds})

# Сохраните DataFrame в CSV файл
predictions_df.to_csv('final_pred.csv', index=False)

# Использование эвристики

In [ ]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Модель 1: ранжирование по вероятностям
В модели для ранжирования кодов я учитываю временной компонент, то есть более новым транзакциям в последовательности даю больший вес за счет учета их позиций. К примеру, для входной последовательности `[1, 1, 1, 1, 1, 2, 2, 2]` выходная последовательность будет начинаться на `[2, 1 ...]`, поскольку сумма позиций для двойки (6+7+8=21) будет больше, чем для единицы (1+2+3+4+5=15)

In [ ]:
# Определяю общий топ-10 кодов по их доле во всем корпусе транзакций
total_top_10 = (df_train['Data'] + df_train['Target'] + df_test['Data']).explode().value_counts(ascending=False).index[:10]
# Длинные последовательности транзакций я буду использовать не целиком,
# поэтому определю максимальную длину исходя из среднего значения и подобранного коэффициента
alpha = 1.9
cutoff = int(np.mean([len(x) for x in df_test['Data']]) / alpha)

In [ ]:
def tops_by_proba(seq, cutoff=cutoff, drop_from=5):
    seq = seq[-cutoff:]
    seq_len = len(seq)
    # Считаю сумму всех индексов в последовательности кодов, индексация в этом случае начинается с 1
    pos_sum = seq_len * (seq_len+1) / 2
    probas = {}

    for code in np.unique(seq):
        # Определяю индексы(позиции), на которых код встречается в последовательности
        positions = np.where(np.array(seq) == code)[0] + 1
        if len(positions) >= drop_from:
            # Вероятность кода для последовательности определяю как отношение суммы его позиций к сумме всех индексов
            probas[code] = sum(positions) / pos_sum
    
    output = sorted(probas, key=probas.get, reverse=True)
    
    if len(output) < 10:
        # Добавляю коды из общего топ-10, если выходная последовательность получилась меньше 10
        output += [x for x in total_top_10 if x not in output]

    return output[:10]

In [ ]:
df_train['tops_by_proba'] = df_train['Data'].apply(tops_by_proba, drop_from=2)
print(mapk(df_train['Target'], df_train['tops_by_proba']))

0.336321564551694


In [ ]:
df_test['Predicted'] = df_test['Data'].apply(tops_by_proba, drop_from=2)

submission = df_test[['Id', 'Predicted']]
submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
submission.to_csv('submission_tops_by_proba_final_2.csv', index=False)

## Модель 2: аналогична Модели 1, но в последовательности не используются повторы.

In [ ]:
def seq_by_weighted_proba_no_repeats(seq):
    seq_len = len(seq)
    # Считаю сумму всех индексов в последовательности кодов, индексация в этом случае начинается с 1
    pos_sum = seq_len * (seq_len+1) / 2
    probas = pd.Series(index=np.unique(seq))

    for code in probas.index:
        # Определяю индексы(позиции), на которых код встречается в последовательности
        positions = np.where(np.array(seq) == code)[0] + 1
        # Вероятность кода для последовательности определяю как отношение суммы его позиций к сумме всех индексов
        probas[code] = sum(positions) / pos_sum
    # Для построения выходной последовательности считаю количество повторений кодов исходя из их вероятностей
    counts = round(probas.sort_values(ascending=False) * 10)
    # Заношу в отдельный список коды с около-нулевой вероятностью
    outsider_codes = list(counts[counts == 0].index)
    output = [None]

    while counts.sum() != 0:
        # Итерируюсь по отсортированному списку кодов, чтобы по возможности избежать повторений
        for code in counts.index:
            if counts[code] != 0:
                # Заношу в выходную последовательность код, если ему не предшествует тот же код, или если кончились аутсайдеры
                if code != output[-1] or not outsider_codes:
                    output.append(code)
                    # Для занесенного кода вычитаю 1 из количества его повторений
                    counts[code] -= 1
                else:
                    # Если код "хочет" повториться, но есть аутсайдеры - вынимаю из списка самого вероятного аутсайдера
                    output.append(outsider_codes.pop(0))
    # Получившуюся последовательность дополняю оставшимися аутсайдерами
    output += outsider_codes
    # Если длина последовательности меньше 10 (без учета None в начале) - дополняю ее самым вероятным кодом до длины 10
    output += [counts.index[0]] * (11 - len(output))

    return output[1:11]

In [ ]:
df_train['seq_by_weighted_proba_no_repeats'] = df_train['Data'].apply(seq_by_weighted_proba_no_repeats)
print(mapk(df_train['Target'], df_train['seq_by_weighted_proba_no_repeats']))

0.3041419814525175


In [ ]:
df_test['Predicted'] = df_test['Data'].apply(seq_by_weighted_proba_no_repeats)

submission = df_test[['Id', 'Predicted']]
submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
submission.to_csv('submission_seq_by_weighted_proba_no_repeats_final.csv', index=False)